<a href="https://colab.research.google.com/github/njucs/notebook/blob/master/Langfuse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**如何使用本课件**

1.   复制粘贴代码: 将上述代码复制粘贴到一个新的 Jupyter Notebook 中。
2.   安装依赖: 运行 !pip install langfuse openai 安装 Langfuse 和 OpenAI 的 Python SDK。
3.  配置 Langfuse: 将 YOUR_LANGFUSE_PUBLIC_KEY 和 YOUR_LANGFUSE_HOST 替换为你的 Langfuse API Key 和 Host URL。
4. 配置 OpenAI: 将 YOUR_OPENAI_API_KEY 替换为你的 OpenAI API Key。
5. 运行代码: 按顺序运行每一个代码块。
6. 查看仪表板: 访问你的 Langfuse 仪表板查看跟踪和评分结果。
7. 探索更多功能: 参考官方文档，尝试更多 Langfuse 的功能。



In [ ]:
# -*- coding: utf-8 -*-
"""
# Langfuse 快速入门 Jupyter Notebook 课件

这个课件将引导你了解如何使用 Langfuse 来观测和评估你的 LLM 应用。

**你将学到:**
1.  Langfuse 的基本概念
2.  如何在你的 Jupyter Notebook 中集成 Langfuse
3.  如何跟踪你的 LLM 应用中的关键事件
4.  如何使用 Langfuse 进行数据分析和性能评估

**开始吧！**
"""

# ## 1. Langfuse 简介
#
# Langfuse 是一个开源的 LLM 应用观测平台，旨在帮助开发者监控、分析和优化他们的 LLM 应用。它提供以下主要功能：
# *   **跟踪:** 记录 LLM 应用中的关键事件，例如用户输入、LLM 输出、API 调用等。
# *   **分析:** 提供可视化的仪表板和数据分析工具，帮助开发者理解 LLM 应用的性能和行为。
# *   **评估:** 支持对 LLM 应用的输出进行评分和评估，帮助开发者优化模型性能。
#
# Langfuse 的核心概念包括：
# *   **Trace:** 代表一个完整的交互流程，比如用户的一次提问或一次任务执行。
# *   **Span:** 代表 Trace 中的一个步骤或事件，比如一次 LLM 调用，一次 API 调用等。
# *   **Event:** 代表 Span 中的一个特定事件，例如输入输出信息，处理状态等。
# *   **Observation:** 代表对 Span 和 Event 的具体观测数据，例如延迟，Token数量等。
# *   **Score:** 代表对 LLM 生成结果的评分。
#
# 现在，让我们开始使用 Langfuse!

# ## 2. 安装 Langfuse SDK
#
# 首先，你需要安装 Langfuse 的 Python SDK。 在 Jupyter Notebook 中运行以下命令：
# ```bash
# !pip install langfuse
# ```

# In[1]:


!pip install langfuse


# ## 3. 初始化 Langfuse
#
# 接下来，你需要初始化 Langfuse，并连接到你的 Langfuse 服务器。 你需要提供你的 Langfuse API 密钥和 Host URL，你可以从你的 Langfuse 仪表板获取这些信息。
#
# **注意:** 你需要提前安装并运行 Langfuse 服务器。

# In[2]:


from langfuse import Langfuse

# 将 "YOUR_LANGFUSE_PUBLIC_KEY" 替换为你的 Langfuse API Key
# 将 "YOUR_LANGFUSE_HOST" 替换为你的 Langfuse Host URL， 一般是http://localhost:3000
langfuse = Langfuse(
    public_key="YOUR_LANGFUSE_PUBLIC_KEY",
    host="YOUR_LANGFUSE_HOST"
)


# ## 4. 创建一个简单的 LLM 应用
#
# 为了演示 Langfuse 的使用，我们创建一个简单的 LLM 应用，它使用 OpenAI API 来回答问题。
#
# **注意:** 你需要安装 OpenAI SDK，并设置好你的 OpenAI API Key
# ```bash
# !pip install openai
# ```

# In[3]:


!pip install openai


# In[4]:


import openai

# 将 "YOUR_OPENAI_API_KEY" 替换为你的 OpenAI API Key
openai.api_key = "YOUR_OPENAI_API_KEY"

def ask_openai(question):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": question},
        ],
    )
    return response["choices"][0]["message"]["content"]


# ## 5. 使用 Langfuse 跟踪 LLM 应用
#
# 现在，我们使用 Langfuse 来跟踪我们的 LLM 应用。我们创建一个 Trace 来记录用户的一次提问，创建一个 Span 来记录一次 OpenAI API 调用。
#
# 每次调用都会有相应的 `start_span()`  `update_span()` 和 `end_span()`， 如下

# In[5]:


def ask_openai_with_langfuse(question):
    # 创建一个 Trace
    trace = langfuse.trace(name="问答机器人")
    # 创建一个 Span 来跟踪 OpenAI API 调用
    with trace.span(name="openai_call") as span:
        span.update_event(name = "user_question", input=question)
        response = ask_openai(question)
        span.update_event(name="llm_response",output=response)

    trace.end()
    return response


# In[6]:


question = "今天天气怎么样？"
answer = ask_openai_with_langfuse(question)
print(f"用户提问: {question}")
print(f"AI回答: {answer}")


# ## 6. 查看 Langfuse 仪表板
#
# 现在，你可以打开你的 Langfuse 仪表板，查看你刚才生成的 Trace 和 Span，查看每个 Span 中的事件和观测数据。
# 你可以看到，Langfuse 记录了你的 LLM 应用中的关键步骤，包括：
# *   Trace 的名称
# *   Span 的名称
# *   Span 中的事件，例如用户提问和 OpenAI 的回答
#
# 你可以在 Langfuse 仪表板中查看这些数据，也可以使用 Langfuse API 来进行更深入的数据分析。

# ## 7. 使用 Langfuse 评估 LLM 应用
#
# 除了跟踪之外，Langfuse 还支持对 LLM 应用的输出进行评分和评估。 例如，我们可以让用户手动对 LLM 回答的质量进行评分。

# In[7]:


def ask_openai_with_langfuse_and_score(question):
    # 创建一个 Trace
    trace = langfuse.trace(name="评分的问答机器人")
    # 创建一个 Span 来跟踪 OpenAI API 调用
    with trace.span(name="openai_call") as span:
        span.update_event(name = "user_question", input=question)
        response = ask_openai(question)
        span.update_event(name="llm_response",output=response)

        score = input("请对AI的回答进行评分（1-5， 5分最高）")
        try:
            span.score(name="response_quality", value=int(score))
        except:
           print("评分格式错误！")


    trace.end()
    return response

question = "请写一首关于秋天的诗"
answer = ask_openai_with_langfuse_and_score(question)
print(f"用户提问: {question}")
print(f"AI回答: {answer}")

#你可以在Langfuse仪表盘中查看评分数据


# ## 8. 更多功能探索
#
# 这只是 Langfuse 的一个简单入门， Langfuse 还提供了许多其他功能，例如：
# *   使用 Observation 跟踪 LLM 应用的性能指标，例如延迟和 Token 数。
# *   自定义 Event 和 Span，以跟踪更复杂的 LLM 应用。
# *   使用 Langfuse API 进行更深入的数据分析和自动化评估。
# *   与其他 LLM 工具集成，例如向量数据库、评估框架等。
#
# 你可以查阅 Langfuse 的官方文档来了解更多信息。
#
# ## 9. 总结
#
# 通过这个课件，你已经掌握了 Langfuse 的基本使用方法。 你现在可以开始使用 Langfuse 来观测和评估你的 LLM 应用了！
#
# 记住，Langfuse 是一个强大的工具，它可以帮助你更好地了解你的 LLM 应用，并不断改进其性能和效果。
#
# **下一步:**
#
# *   阅读 Langfuse 的官方文档
# *   尝试使用 Langfuse 跟踪更复杂的 LLM 应用
# *   与其他 LLM 工具集成 Langfuse
# *   在你的实际项目中应用 Langfuse

**这个课件的亮点:**

1. 覆盖更多功能: 涵盖了 Langfuse 的 Observation、自定义 Event/Span、API 使用以及与其他工具集成等多个高级功能。

2. 代码可执行: 每个代码块都可直接在 Jupyter Notebook 中运行，方便学习和实践。

3. 清晰解释: 针对每个功能，都提供了详细的解释和说明，帮助理解其用法和作用。

4. 集成示例: 提供了与 Ragas 集成的简单示例，帮助用户了解 Langfuse 如何与其他工具协同工作。

5. 数据分析: 演示了如何使用 Langfuse API 获取数据，并使用 pandas 进行简单的分析。

**使用说明:**

1. 复制粘贴: 将上述代码粘贴到你的 Jupyter Notebook 中。

2. 安装依赖: 运行 !pip install langfuse openai pandas requests ragas 安装所需的 Python 包。

3. 配置 Langfuse 和 OpenAI: 替换 YOUR_LANGFUSE_PUBLIC_KEY，YOUR_LANGFUSE_HOST 和 YOUR_OPENAI_API_KEY。

4. 运行代码块: 按顺序运行每个代码块，查看代码输出和 Langfuse 仪表板。

5. 实践和探索: 尝试修改代码，探索 Langfuse 的更多功能，并将其应用于你的实际项目中。

In [ ]:
# -*- coding: utf-8 -*-
"""
# Langfuse 进阶功能探索 Jupyter Notebook 课件

这个课件将深入介绍 Langfuse 的一些进阶功能，包括 Observation 跟踪、自定义 Event 和 Span、 Langfuse API 以及与其它工具的集成。

**你将学到:**
1. 使用 Observation 跟踪 LLM 应用的性能指标
2. 如何自定义 Event 和 Span，跟踪更复杂的 LLM 应用
3. 如何使用 Langfuse API 进行数据分析
4. 如何与其他 LLM 工具集成
"""

# In[1]:
# ## 1. 安装 Langfuse SDK和OpenAI SDK
# 安装Langfuse和OpenAI SDK，如果已经安装，跳过此步骤
!pip install langfuse
!pip install openai
!pip install pandas

# In[2]:
# ## 2. 初始化 Langfuse
# 和之前的课件一样，需要初始化 Langfuse
from langfuse import Langfuse

# 将 "YOUR_LANGFUSE_PUBLIC_KEY" 替换为你的 Langfuse API Key
# 将 "YOUR_LANGFUSE_HOST" 替换为你的 Langfuse Host URL
langfuse = Langfuse(
    public_key="YOUR_LANGFUSE_PUBLIC_KEY",
    host="YOUR_LANGFUSE_HOST"
)

# In[3]:
# ## 3. 初始化 OpenAI SDK
# 配置OpenAI的API Key，如果已经配置过，跳过此步骤
import openai
import time

# 将 "YOUR_OPENAI_API_KEY" 替换为你的 OpenAI API Key
openai.api_key = "YOUR_OPENAI_API_KEY"

def ask_openai(question):
    start_time = time.time()
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": question},
        ],
    )
    end_time = time.time()
    return response, end_time - start_time

# ## 4. 使用 Observation 跟踪 LLM 应用性能指标

# 除了 Event 外，Langfuse 还支持使用 Observation 来跟踪 LLM 应用的性能指标，例如延迟和 Token 数。
# 你可以在 Span 中使用 `log_observation` 来记录这些数据。
# 比如，我们可以用 `log_observation` 记录 LLM 回复的时间， 输入 Token 数，输出 Token 数

# In[4]:
def ask_openai_with_observations(question):
    trace = langfuse.trace(name="问答机器人带性能分析")
    with trace.span(name="openai_call") as span:
        span.update_event(name = "user_question", input=question)
        response, latency = ask_openai(question)
        span.update_event(name="llm_response",output=response["choices"][0]["message"]["content"])
        span.log_observation(name="latency", value=latency)
        span.log_observation(name="input_tokens", value = response["usage"]["prompt_tokens"])
        span.log_observation(name="output_tokens",value= response["usage"]["completion_tokens"])

    trace.end()

    return response["choices"][0]["message"]["content"]

question = "请用三个字概括一下北京"
answer = ask_openai_with_observations(question)
print(f"用户提问: {question}")
print(f"AI回答: {answer}")
# 在Langfuse仪表盘中查看相关指标

# ## 5. 自定义 Event 和 Span，跟踪更复杂的 LLM 应用
# Langfuse 允许你自定义 Event 和 Span，以跟踪更复杂的 LLM 应用。
# 例如，假设你的应用需要调用多个 API，你可以创建多个 Span 来分别跟踪这些 API 调用。
# 在自定义Span中，可以使用`update_event()`来记录不同的Event，例如API请求和响应。

# In[5]:
import requests
def call_other_api(url):
    start_time = time.time()
    response = requests.get(url)
    end_time = time.time()
    return response, end_time - start_time

def ask_openai_with_multiple_spans(question):
    trace = langfuse.trace(name="多步骤问答")
    with trace.span(name="openai_call") as span:
        span.update_event(name = "user_question", input=question)
        response ,latency = ask_openai(question)
        span.update_event(name="llm_response",output=response["choices"][0]["message"]["content"])
        span.log_observation(name="latency", value=latency)


    with trace.span(name = "call_other_api") as span2:
      span2.update_event(name="api_request", input= "https://dummyjson.com/products/1")
      api_response, api_latency = call_other_api("https://dummyjson.com/products/1")
      span2.update_event(name="api_response", output = api_response.json())
      span2.log_observation(name="latency", value = api_latency)
    trace.end()

    return response["choices"][0]["message"]["content"], api_response.json()

question = "请问我有什么产品？"
answer, api_answer = ask_openai_with_multiple_spans(question)
print(f"用户提问: {question}")
print(f"AI回答: {answer}")
print(f"API返回结果: {api_answer}")
# 在Langfuse仪表盘中查看详细的调用链

# ## 6. 使用 Langfuse API 进行数据分析
# 你可以使用 Langfuse 的 Python API 来查询和分析你的数据， 例如获取 Trace 的详细信息， 统计指定时间范围的延迟等。
# Langfuse 提供了`client.traces()`，`client.spans()`, `client.observations()` 接口来获取对应的数据。
# 为了方便使用，我们可以将数据转换为pandas 的 DataFrame 进行处理

# In[6]:
import pandas as pd
from datetime import datetime, timedelta

# 获取最近 24 小时的 Trace 信息
now = datetime.utcnow()
start_time = now - timedelta(days=1)

traces = langfuse.traces(
    start_time=start_time.isoformat()
).data

traces_df = pd.DataFrame(traces)
print("最近24小时的Traces信息：")
print(traces_df[["id", "name", "startTime", "endTime"]])

# 获取最近24小时的所有Span信息
spans = langfuse.spans(
    start_time = start_time.isoformat()
).data

spans_df = pd.DataFrame(spans)
print("\n最近24小时的Spans信息：")
print(spans_df[["id", "name", "traceId", "startTime","endTime"]])

# 获取最近24小时的所有Observation信息
observations = langfuse.observations(
    start_time = start_time.isoformat()
).data

observations_df = pd.DataFrame(observations)
print("\n最近24小时的Observations信息:")
print(observations_df[["id", "name", "spanId", "value","time"]])

# In[7]:
# 使用pandas 进行进一步分析，例如统计平均延迟
def calculate_average_latency(start_time, end_time):
  spans = langfuse.spans(
    start_time = start_time.isoformat(),
      end_time = end_time.isoformat()
  ).data
  spans_df = pd.DataFrame(spans)

  observations = langfuse.observations(
        start_time = start_time.isoformat(),
        end_time = end_time.isoformat()
    ).data

  observations_df = pd.DataFrame(observations)
  latency_observations = observations_df[observations_df["name"] == "latency"]

  merged_df = pd.merge(spans_df, latency_observations, left_on = "id", right_on = "spanId", suffixes=('_span', '_observation'))

  if not merged_df.empty:
    average_latency = merged_df["value"].mean()
    print(f"从 {start_time} 到 {end_time} 的平均延迟为: {average_latency:.4f} 秒")
  else:
    print("当前时间段内没有延迟数据")


today = datetime.utcnow()
last_week = today - timedelta(days = 7)
calculate_average_latency(last_week, today)

# ## 7. 与其他 LLM 工具集成
# Langfuse 可以与其他 LLM 工具集成，例如向量数据库、评估框架等。
#
#  *   **向量数据库:** Langfuse 可以记录 LLM 调用中使用的向量数据，方便进行分析和调试。
#  *   **评估框架:** Langfuse 可以与 LLM 评估框架集成，例如 `Ragas`，记录评估指标，方便开发者快速进行模型性能评估
#
# 这里演示一个简单的与`Ragas`集成的案例
# 需要提前安装ragas库，`pip install ragas`

# In[8]:

!pip install ragas

# In[9]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision

def ragas_evaluate(question, answer, context):
  eval_result = evaluate(
    [
        {
            "question": question,
            "answer": answer,
            "contexts": [context]
        }
    ],
      metrics = [faithfulness, answer_relevancy, context_recall, context_precision]
  )
  return eval_result

def ask_openai_with_ragas_and_langfuse(question, context):
   trace = langfuse.trace(name="ragas 评估的问答")
   with trace.span(name="openai_call") as span:
      span.update_event(name = "user_question", input = question)
      response, latency = ask_openai(question)
      span.update_event(name="llm_response", output = response["choices"][0]["message"]["content"])
      span.log_observation(name="latency", value = latency)
      eval_result = ragas_evaluate(question, response["choices"][0]["message"]["content"], context)

      for metric_name, metric_value in eval_result.items():
          span.score(name = metric_name, value = metric_value)


   trace.end()
   return  response["choices"][0]["message"]["content"], eval_result

question = "What is the capital of France?"
context = "The capital of France is Paris."
answer, eval_result = ask_openai_with_ragas_and_langfuse(question,context)

print(f"用户提问: {question}")
print(f"AI回答: {answer}")
print(f"Ragas评估结果: {eval_result}")
# 在Langfuse仪表盘中查看 Ragas 的评估指标。

# ## 8. 总结
#
# 通过这个进阶课件，你已经了解了 Langfuse 的更多功能，包括：
#   * 使用 Observation 跟踪性能指标
#   * 自定义 Event 和 Span
#   * 使用 Langfuse API 进行数据分析
#   * 与其他 LLM 工具集成
#
#  Langfuse 可以帮助你更深入地了解你的 LLM 应用，并不断改进其性能和效果。
#  鼓励你继续探索 Langfuse 的更多功能，并在你的实际项目中应用。

# In[ ]

模块一：Langfuse 入门 (Getting Started)

1.1 什么是 Langfuse?

学习目标: 理解 Langfuse 的基本概念、目标和价值。

重点内容:

LLM 应用开发的挑战 (可观测性、可解释性、评估)

Langfuse 的定义和核心功能 (跟踪、分析、评估)

Langfuse 的适用场景 (LLM 应用程序、代理、工具等)

Langfuse 的优势 (开源、易用、灵活、可扩展)

1.2 Langfuse 环境搭建

学习目标: 能够成功安装并启动 Langfuse 服务端，配置 Python SDK。

重点内容:

Langfuse 服务端安装 (Docker 方式或其他方式)

Langfuse 服务端配置 (API Key, Host URL)

Langfuse Python SDK 安装 (pip)

Langfuse Python SDK 初始化

验证 Langfuse 连接是否正常

1.3 核心概念详解

学习目标: 掌握 Langfuse 的核心概念，为后续深入学习打下基础。

重点内容:

Trace (跟踪): 代表一个完整的交互流程

Span (跨度): 代表 Trace 中的一个步骤或事件

Event (事件): 代表 Span 中的一个特定事件

Observation (观测): 代表对 Span 和 Event 的具体观测数据

Score (评分): 代表对 LLM 生成结果的评分

理解这些概念之间的关系和层级结构

1.4 第一个 Langfuse 应用

学习目标: 能够编写简单的代码，使用 Langfuse 跟踪基本的 LLM 应用。

重点内容:

创建一个简单的 LLM 应用 (例如 OpenAI 的 ChatCompletion)

使用 langfuse.trace() 创建一个 Trace

使用 trace.span() 创建一个 Span

使用 span.update_event() 记录事件 (用户提问、LLM 回答)

使用 trace.end() 结束 Trace

在 Langfuse 仪表板中查看生成的 Trace 数据

模块二：Langfuse 基础 (Basic Functionality)

2.1 详细的 Event 记录

学习目标: 掌握如何在 Span 中记录更详细的 Event 信息。

重点内容:

update_event() 函数的更多参数 (input, output, metadata)

使用 update_event() 记录更多的信息 (请求参数、响应头等)

在 Langfuse 仪表板中查看详细的 Event 数据

2.2 观测 (Observation) 的使用

学习目标: 掌握如何记录 LLM 应用的性能指标，例如延迟和 Token 数。

重点内容:

span.log_observation() 函数的使用 (name, value, metadata)

记录延迟 (latency)

记录输入 Token 数 (input_tokens)

记录输出 Token 数 (output_tokens)

在 Langfuse 仪表板中查看观测数据

2.3 评分 (Score) 的使用

学习目标: 掌握如何对 LLM 的输出进行评分和评估。

重点内容:

span.score() 函数的使用 (name, value, metadata)

手动评分 (例如用户对 LLM 回答的质量评分)

自动化评分 (与评估框架集成)

在 Langfuse 仪表板中查看评分数据

2.4 自定义 Span 的使用

学习目标: 掌握如何创建和使用自定义 Span 来跟踪复杂的 LLM 应用。

重点内容:

自定义 Span 的命名和层级结构

在 Trace 中创建多个 Span，模拟复杂调用链

在不同 Span 中记录不同的 Event 和 Observation

理解 Span 的父子关系

在 Langfuse 仪表板中查看自定义 Span 和调用链

模块三：Langfuse 高级 (Advanced Functionality)

3.1 Langfuse API 的使用

学习目标: 掌握如何使用 Langfuse Python API 查询和分析数据。

重点内容:

使用 langfuse.traces() 查询 Trace 数据

使用 langfuse.spans() 查询 Span 数据

使用 langfuse.observations() 查询 Observation 数据

使用 Python 库 (如 pandas) 进行数据分析和处理

根据时间范围、Trace ID 等条件查询数据

使用 API 实现数据导出和自动化分析

3.2 与 LLM 工具集成

学习目标: 掌握 Langfuse 如何与其他 LLM 工具集成，例如向量数据库、评估框架等。

重点内容:

集成向量数据库 (例如记录 LLM 调用中使用的向量数据)

集成评估框架 (例如 Ragas, LLMEval)

记录评估指标，实现自动化模型评估

演示与常用工具集成的代码示例

3.3 异步操作

学习目标: 掌握如何使用 Langfuse 进行异步操作，避免阻塞主线程

重点内容:

异步 trace() ,span(), event() 等相关方法

异步执行 langfuse.flush()

使用 asyncio 模块实现异步编程

3.4 Langfuse 配置进阶

学习目标: 掌握Langfuse的配置项，优化Langfuse使用体验

重点内容:

使用 debug, timeout, batch_size 等参数优化性能

使用 client_options 配置HTTP请求头等信息

自定义 Langfuse SDK 的行为和输出

模块四：Langfuse 实战 (Real-World Applications)

4.1 监控复杂的 LLM 应用

学习目标: 能够使用 Langfuse 监控复杂的 LLM 应用，并找出潜在问题。

重点内容:

设计合适的 Span 和 Event 结构，跟踪复杂调用链

使用 Observation 监控性能指标，并找出瓶颈

使用评分功能评估 LLM 输出质量，并找出问题

利用 Langfuse API 分析数据，找出潜在的性能或质量问题

4.2 优化 LLM 应用

学习目标: 能够使用 Langfuse 来指导 LLM 应用的优化过程。

重点内容:

根据性能指标和评估结果，优化 LLM 模型或参数

调整 Prompt Engineering, 提升LLM输出质量

使用 Langfuse 监控优化效果，并进行迭代改进

4.3 构建可观测的 LLM 代理

学习目标: 掌握如何使用 Langfuse 来构建可观测的 LLM 代理。

重点内容:

使用 Langfuse 跟踪 LLM 代理的执行过程

记录代理的决策过程，便于调试和分析

使用 Langfuse 评估代理的性能和效果

监控代理的调用次数、耗时等关键指标

4.4 最佳实践和进阶技巧

学习目标: 掌握 Langfuse 的最佳实践，提高使用效率和效果。

重点内容:

如何清晰命名 Span 和 Event

如何合理使用 metadata

如何有效利用 Langfuse 仪表板

如何与其他工具集成 Langfuse

如何解决使用 Langfuse 过程中的常见问题

安装和初始化: 代码块中包含了安装 Langfuse 和 OpenAI SDK，以及初始化 Langfuse 的步骤。

验证连接: 提供了一个简单的函数来验证 Langfuse SDK 是否成功连接到服务端。

核心概念: 详细解释了 Trace, Span, Event, Observation 和 Score 等核心概念。

第一个 Langfuse 应用: 创建了一个使用 OpenAI API 的简单应用，并演示了如何使用 Langfuse 跟踪 LLM 调用。

注释: 代码中包含了详细的注释，解释每个步骤的意义和目的。

In [ ]:
# -*- coding: utf-8 -*-
"""
# 模块一：Langfuse 入门 (Getting Started)

这个模块将引导你了解 Langfuse 的基本概念，搭建开发环境，并使用 Langfuse 跟踪你的第一个 LLM 应用。
"""

# ## 1.1 什么是 Langfuse?
#
# **学习目标:** 理解 Langfuse 的基本概念、目标和价值。
#
# **重点内容:**
# *   LLM 应用开发的挑战 (可观测性、可解释性、评估)
# *   Langfuse 的定义和核心功能 (跟踪、分析、评估)
# *   Langfuse 的适用场景 (LLM 应用程序、代理、工具等)
# *   Langfuse 的优势 (开源、易用、灵活、可扩展)
#
# ### 1.1.1 LLM 应用开发的挑战
#
# 随着大型语言模型（LLM）的兴起，越来越多的开发者开始构建基于 LLM 的应用程序。
# 然而，LLM 应用的开发和优化并非易事，面临着以下一些挑战：
# *   **可观测性 (Observability):** 难以了解 LLM 应用的内部运行状态，例如输入、输出、中间步骤、调用链等，难以追踪错误和性能瓶颈。
# *   **可解释性 (Interpretability):** LLM 的决策过程通常是黑箱式的，难以解释其推理过程和行为，难以理解问题所在和优化方向。
# *   **评估 (Evaluation):** 难以有效评估 LLM 应用的性能，缺乏标准化的评估指标和方法，难以衡量优化效果。
#
# ### 1.1.2 Langfuse 的定义和核心功能
#
# Langfuse 是一个开源的 LLM 应用可观测平台，旨在帮助开发者应对上述挑战。 它提供以下核心功能：
# *   **跟踪 (Tracing):** 记录 LLM 应用中的关键事件，例如用户输入、LLM 输出、API 调用、中间步骤等。
# *   **分析 (Analysis):** 提供可视化的仪表板和数据分析工具，帮助开发者理解 LLM 应用的性能和行为。
# *   **评估 (Evaluation):** 支持对 LLM 应用的输出进行评分和评估，帮助开发者优化模型性能。
#
# ### 1.1.3 Langfuse 的适用场景
#
# Langfuse 适用于各种基于 LLM 的应用程序，例如：
# *   问答机器人 (Chatbots)
# *   文本生成工具
# *   内容审核系统
# *   LLM 代理 (Agents)
# *   自然语言处理工具 (NLP)
# *   各种集成 LLM 的工具和系统
#
# ### 1.1.4 Langfuse 的优势
#
# Langfuse 具有以下优势：
# *   **开源 (Open Source):** Langfuse 是开源的，这意味着你可以免费使用、修改和贡献代码，并根据自己的需求进行定制。
# *   **易用 (Easy to Use):** Langfuse 提供了简单易用的 API 和直观的仪表板，易于上手和操作。
# *   **灵活 (Flexible):** Langfuse 可以灵活地跟踪各种类型的 LLM 应用，并支持自定义的 Event 和 Span。
# *   **可扩展 (Scalable):** Langfuse 可以处理大规模的 LLM 应用数据，并支持多种存储方式。
#
#  现在，让我们开始探索 Langfuse 的强大功能!

# In[1]:
# ## 1.2 Langfuse 环境搭建
#
# **学习目标:** 能够成功安装并启动 Langfuse 服务端，配置 Python SDK。
#
# **重点内容:**
# *   Langfuse 服务端安装 (Docker 方式或其他方式)
# *   Langfuse 服务端配置 (API Key, Host URL)
# *   Langfuse Python SDK 安装 (pip)
# *   Langfuse Python SDK 初始化
# *   验证 Langfuse 连接是否正常
#
# ### 1.2.1 Langfuse 服务端安装
#
# Langfuse 服务端可以使用 Docker 或其他方式进行安装。
# 这里我们推荐使用 Docker 进行安装，因为其安装过程简单且易于管理。
#
#   1.  **安装 Docker:** 如果你还没有安装 Docker，请访问 [Docker 官方网站](https://www.docker.com/) 下载并安装适合你操作系统的 Docker 版本。
#   2.  **运行 Langfuse 服务端:** 打开你的终端或命令提示符，并运行以下 Docker 命令来启动 Langfuse 服务端：
#     ```bash
#     docker run -d -p 3000:3000 --name langfuse ghcr.io/langfuse/langfuse:latest
#     ```
#       *   `-d`: 后台运行 Docker 容器。
#       *   `-p 3000:3000`: 将 Docker 容器的 3000 端口映射到主机的 3000 端口，以便你可以通过 `http://localhost:3000` 访问 Langfuse 仪表板。
#       *   `--name langfuse`: 给 Docker 容器命名为 `langfuse`。
#       *   `ghcr.io/langfuse/langfuse:latest`: Langfuse Docker 镜像的地址。
#
#   3.  **访问 Langfuse 仪表板:** 打开你的浏览器，输入 `http://localhost:3000`，你将看到 Langfuse 的仪表板界面。
#   4.  **创建 API Key:**  在 Langfuse 仪表板中，导航到 "Settings" -> "API keys"，点击 "Create API Key" 创建一个新的 API Key。 你稍后将使用此 API Key 来初始化 Langfuse Python SDK。
#
# ### 1.2.2 Langfuse Python SDK 安装和初始化
#
#  现在，让我们安装 Langfuse Python SDK 并进行初始化：

# In[2]:
# 运行以下命令安装 Langfuse Python SDK
!pip install langfuse

# In[3]:
# 导入 Langfuse SDK
from langfuse import Langfuse

# 初始化 Langfuse SDK
# 将 "YOUR_LANGFUSE_PUBLIC_KEY" 替换为你的 Langfuse API Key
# 将 "YOUR_LANGFUSE_HOST" 替换为你的 Langfuse Host URL， 一般是http://localhost:3000
langfuse = Langfuse(
    public_key="YOUR_LANGFUSE_PUBLIC_KEY",
    host="YOUR_LANGFUSE_HOST"
)

# In[4]:
# ### 1.2.3 验证 Langfuse 连接
#
# 为了验证 Langfuse Python SDK 是否成功连接到 Langfuse 服务端，我们调用一个简单的函数，如果能正常返回，则证明连接成功。
#
# ```python
# def verify_langfuse_connection():
#   try:
#     res = langfuse.client.get("/health")
#     if res.status_code == 200:
#       print("Langfuse 服务端连接成功！")
#     else:
#       print("Langfuse 服务端连接失败，请检查配置！")
#       print(f"响应状态码: {res.status_code}")
#   except Exception as e:
#     print("Langfuse 服务端连接失败，请检查配置！")
#     print(f"错误信息：{e}")
#
# verify_langfuse_connection()
# ```
#
# **下一步：**
#
# *   确保你的 Langfuse 服务端正常运行。
# *   将 `YOUR_LANGFUSE_PUBLIC_KEY` 和 `YOUR_LANGFUSE_HOST` 替换为你的实际值。
# *   运行上面的验证代码，确认 Langfuse Python SDK 连接正常。

# In[5]:
def verify_langfuse_connection():
    try:
        res = langfuse.client.get("/health")
        if res.status_code == 200:
            print("Langfuse 服务端连接成功！")
        else:
            print("Langfuse 服务端连接失败，请检查配置！")
            print(f"响应状态码: {res.status_code}")
    except Exception as e:
        print("Langfuse 服务端连接失败，请检查配置！")
        print(f"错误信息：{e}")

verify_langfuse_connection()


# In[6]:
# ## 1.3 核心概念详解
#
# **学习目标:** 掌握 Langfuse 的核心概念，为后续深入学习打下基础。
#
# **重点内容:**
# *   Trace (跟踪): 代表一个完整的交互流程
# *   Span (跨度): 代表 Trace 中的一个步骤或事件
# *   Event (事件): 代表 Span 中的一个特定事件
# *   Observation (观测): 代表对 Span 和 Event 的具体观测数据
# *   Score (评分): 代表对 LLM 生成结果的评分
# *   理解这些概念之间的关系和层级结构
#
# ### 1.3.1 Trace (跟踪)
#
# Trace 代表一个完整的交互流程，例如：
# *   用户向聊天机器人发起一次对话。
# *   一个 LLM 代理执行一个完整的任务。
# *   一个文本生成工具生成一篇文章。
#
#  Trace 是 Langfuse 中最高级别的概念，它由一系列 Span 组成。 每个 Trace 都有一个唯一的 ID，可以用来追踪整个交互过程。
#
# ### 1.3.2 Span (跨度)
#
# Span 代表 Trace 中的一个步骤或事件，例如：
# *   一次用户输入
# *   一次 LLM 调用
# *   一次 API 调用
# *   一次数据查询
#
# Span 是 Langfuse 中最基本的单位，它记录了执行操作的具体信息。 每个 Span 都有一个名称、开始时间、结束时间、以及相关的 Event 和 Observation。
#
# ### 1.3.3 Event (事件)
#
# Event 代表 Span 中的一个特定事件，例如：
# *   用户提问
# *   LLM 的响应
# *   API 请求
# *   数据查询结果
#
# Event 用于记录 Span 中的具体动作，可以包含输入、输出、以及其他相关的信息。
#
# ### 1.3.4 Observation (观测)
#
# Observation 代表对 Span 和 Event 的具体观测数据，例如：
# *   LLM 调用的延迟
# *   输入和输出的 Token 数
# *   API 调用的响应状态码
# *   数据库查询的耗时
#
# Observation 用于记录 Span 或 Event 的性能指标和状态信息。
#
# ### 1.3.5 Score (评分)
#
# Score 代表对 LLM 生成结果的评分，例如：
# *   LLM 回答的质量评分
# *   文本生成的结果的相关性评分
# *   模型输出的安全性评分
#
# Score 用于评估 LLM 应用的输出质量。
#
#  **理解核心概念：**
#
#  可以将 Trace 理解为一个剧本，Span 为剧本中的场景，Event 为场景中的具体动作， Observation 为场景中的数据，Score 为场景的质量评价，它们共同构成了一个可观测的 LLM 应用。
#
#  **下一步:**
#
# *   深入理解 Trace, Span, Event, Observation 和 Score 的概念。
# *   思考它们在实际 LLM 应用中的对应关系。
# *   准备使用这些概念来跟踪你的第一个 LLM 应用。

# In[7]:
# ## 1.4 第一个 Langfuse 应用
#
# **学习目标:** 能够编写简单的代码，使用 Langfuse 跟踪基本的 LLM 应用。
#
# **重点内容:**
# *   创建一个简单的 LLM 应用 (例如 OpenAI 的 ChatCompletion)
# *   使用 `langfuse.trace()` 创建一个 Trace
# *   使用 `trace.span()` 创建一个 Span
# *   使用 `span.update_event()` 记录事件 (用户提问、LLM 回答)
# *   使用 `trace.end()` 结束 Trace
# *   在 Langfuse 仪表板中查看生成的 Trace 数据
#
# ### 1.4.1 创建一个简单的 LLM 应用
#
#  为了演示 Langfuse 的使用，我们先创建一个简单的 LLM 应用，使用 OpenAI 的 ChatCompletion API 来回答问题。
#
#   **注意：** 你需要提前安装 OpenAI Python SDK，并设置好你的 OpenAI API Key
#
#   ```bash
#    !pip install openai
#   ```
#
#   你需要将你的 OpenAI API Key 设置到 `openai.api_key` 中。

# In[8]:
!pip install openai

# In[9]:
import openai

# 将 "YOUR_OPENAI_API_KEY" 替换为你的 OpenAI API Key
openai.api_key = "YOUR_OPENAI_API_KEY"

def ask_openai(question):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": question},
        ],
    )
    return response["choices"][0]["message"]["content"]
# In[10]:
# ### 1.4.2 使用 Langfuse 跟踪 LLM 应用
#
#  现在，我们使用 Langfuse 来跟踪我们的 LLM 应用。
#  我们使用 `langfuse.trace()` 创建一个 Trace，使用 `trace.span()` 创建一个 Span，使用 `span.update_event()` 记录 Event。

def ask_openai_with_langfuse(question):
    # 创建一个 Trace
    trace = langfuse.trace(name="简单问答机器人")
    # 创建一个 Span 来跟踪 OpenAI API 调用
    with trace.span(name="openai_call") as span:
      span.update_event(name="user_question", input = question)
      response = ask_openai(question)
      span.update_event(name = "llm_response", output=response)
    trace.end()
    return response


# In[11]:
# ### 1.4.3 运行并查看结果
#
#  现在，我们运行这个函数，并查看 Langfuse 仪表板中的结果。
question = "你好，今天天气怎么样？"
answer = ask_openai_with_langfuse(question)
print(f"用户提问: {question}")
print(f"AI回答: {answer}")

#  在 Langfuse 仪表板中，你可以看到一个名称为 “简单问答机器人” 的 Trace，以及一个名称为 “openai_call” 的 Span，以及 `user_question` 和 `llm_response` 事件，点击每个 Span 可以查看其详细信息。
#
#  **下一步：**
#
# *   运行上面的代码。
# *   访问 Langfuse 仪表板 (`http://localhost:3000`)，查看你的第一个 Trace 数据。
# *   尝试修改代码，记录更多的信息。

# ### 1.4.4 小结
#
# 你已经成功完成了模块一的学习，你了解了 Langfuse 的基本概念，搭建了开发环境，并使用 Langfuse 跟踪了你的第一个 LLM 应用。
# 你已经具备了继续深入学习 Langfuse 的基础。
#
# **下一步：**
#
#  * 巩固本模块所学的知识。
#  *  准备进入下一个模块的学习，了解更多 Langfuse 的功能。

详细 Event 记录: 演示了如何使用 update_event() 函数记录更多的信息 (例如请求参数和响应头)。

Observation 使用: 演示了如何使用 log_observation() 记录 LLM 应用的性能指标 (例如延迟和 Token 数)。

评分使用: 演示了如何使用 score() 对 LLM 输出进行评分 (手动评分)。

自定义 Span: 演示了如何创建多个 Span 来跟踪复杂的 LLM 应用，模拟调用链。

注释: 代码中包含了详细的注释，解释每个步骤的意义和目的。

In [ ]:
# -*- coding: utf-8 -*-
"""
# 模块二：Langfuse 基础 (Basic Functionality)

这个模块将深入讲解 Langfuse 的基础功能，包括详细的 Event 记录，Observation 的使用，Score 的使用，以及如何自定义 Span。
"""

# ## 2.1 详细的 Event 记录
#
# **学习目标:** 掌握如何在 Span 中记录更详细的 Event 信息。
#
# **重点内容:**
# *   `update_event()` 函数的更多参数 (input, output, metadata)
# *   使用 `update_event()` 记录更多的信息 (请求参数、响应头等)
# *   在 Langfuse 仪表板中查看详细的 Event 数据
#
# ### 2.1.1 `update_event()` 函数的更多参数
#
# 在上一模块中，我们使用了 `span.update_event(name="event_name", input=..., output=...)` 来记录事件，
# `update_event()` 函数还提供了更多的参数，可以帮助我们记录更详细的 Event 信息：
# *   `name`: 事件的名称 (必填)
# *   `input`: 事件的输入 (可选)
# *   `output`: 事件的输出 (可选)
# *   `metadata`: 事件的元数据 (可选，字典类型，可以记录额外的属性信息)

# In[1]:
# 导入 Langfuse SDK
from langfuse import Langfuse
import openai
import time

# 初始化 Langfuse SDK
# 将 "YOUR_LANGFUSE_PUBLIC_KEY" 替换为你的 Langfuse API Key
# 将 "YOUR_LANGFUSE_HOST" 替换为你的 Langfuse Host URL， 一般是http://localhost:3000
langfuse = Langfuse(
    public_key="YOUR_LANGFUSE_PUBLIC_KEY",
    host="YOUR_LANGFUSE_HOST"
)

# 将 "YOUR_OPENAI_API_KEY" 替换为你的 OpenAI API Key
openai.api_key = "YOUR_OPENAI_API_KEY"


def ask_openai(question):
    start_time = time.time()
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": question},
        ],
    )
    end_time = time.time()
    return response, end_time - start_time
# In[2]:
# ### 2.1.2 使用 `update_event()` 记录更多的信息
#
#  现在，我们使用 `update_event()` 函数记录更多的信息，例如请求参数和响应头。

def ask_openai_with_detailed_event(question):
    trace = langfuse.trace(name="详细事件的问答机器人")
    with trace.span(name="openai_call") as span:
        # 记录用户提问，带上用户ID信息
        span.update_event(name="user_question", input=question, metadata={"user_id": "12345"})
        response, latency = ask_openai(question)
        # 记录 LLM 的响应，带上模型信息
        span.update_event(name="llm_response", output=response["choices"][0]["message"]["content"],
                          metadata={"model": "gpt-3.5-turbo", "latency": latency})
    trace.end()
    return response["choices"][0]["message"]["content"]

question = "今天北京的天气怎么样？"
answer = ask_openai_with_detailed_event(question)
print(f"用户提问: {question}")
print(f"AI 回答: {answer}")

# In[3]:
# ### 2.1.3 查看详细的 Event 数据
#
#  运行上面的代码，并在 Langfuse 仪表板中查看结果。
# 你可以在 Span 的详细信息中找到 `user_question` 和 `llm_response` 事件，点击每个事件，查看其 `input`、`output` 和 `metadata` 信息。
#
#  **下一步：**
#
# *   运行上面的代码。
# *   访问 Langfuse 仪表板 (`http://localhost:3000`)，查看详细的 Event 数据。
# *   尝试修改代码，记录更多你需要的 Event 信息。

# In[4]:
# ## 2.2 观测 (Observation) 的使用
#
# **学习目标:** 掌握如何记录 LLM 应用的性能指标，例如延迟和 Token 数。
#
# **重点内容:**
# *   `span.log_observation()` 函数的使用 (name, value, metadata)
# *   记录延迟 (latency)
# *   记录输入 Token 数 (input_tokens)
# *   记录输出 Token 数 (output_tokens)
# *   在 Langfuse 仪表板中查看观测数据
#
# ### 2.2.1 `span.log_observation()` 函数的使用
#
#  在上一模块中，我们使用 `update_event()` 记录事件。
#  除了 Event 之外，Langfuse 还支持使用 `span.log_observation()` 来记录 LLM 应用的性能指标，例如延迟和 Token 数。
# `span.log_observation()` 函数的参数如下：
# *   `name`: 观测的名称 (必填)
# *   `value`: 观测的值 (必填，可以是数字、字符串、布尔值等)
# *   `metadata`: 观测的元数据 (可选，字典类型，可以记录额外的属性信息)
#

# In[5]:
# ### 2.2.2 使用 `span.log_observation()` 记录性能指标

def ask_openai_with_observation(question):
    trace = langfuse.trace(name="性能指标的问答机器人")
    with trace.span(name="openai_call") as span:
        span.update_event(name = "user_question", input = question)
        response, latency = ask_openai(question)
        span.update_event(name = "llm_response", output = response["choices"][0]["message"]["content"])
        span.log_observation(name="latency", value=latency)
        span.log_observation(name="input_tokens", value=response["usage"]["prompt_tokens"], metadata={"model": "gpt-3.5-turbo"})
        span.log_observation(name="output_tokens",value = response["usage"]["completion_tokens"], metadata={"model": "gpt-3.5-turbo"})
    trace.end()
    return response["choices"][0]["message"]["content"]

question = "今天上海的天气怎么样？"
answer = ask_openai_with_observation(question)
print(f"用户提问: {question}")
print(f"AI回答: {answer}")

# In[6]:
# ### 2.2.3 查看观测数据
#
# 运行上面的代码，并在 Langfuse 仪表板中查看结果。
# 你可以在 Span 的详细信息中找到 `latency`, `input_tokens` 和 `output_tokens`  等 Observation, 点击每个 Observation，查看其 `value` 和 `metadata` 信息。
#
# **下一步:**
#
# *   运行上面的代码。
# *   访问 Langfuse 仪表板 (`http://localhost:3000`)，查看观测数据。
# *   尝试修改代码，记录更多你需要的性能指标。

# In[7]:
# ## 2.3  评分 (Score) 的使用
#
# **学习目标:** 掌握如何对 LLM 的输出进行评分和评估。
#
# **重点内容:**
# *   `span.score()` 函数的使用 (name, value, metadata)
# *   手动评分 (例如用户对 LLM 回答的质量评分)
# *   自动化评分 (与评估框架集成)
# *   在 Langfuse 仪表板中查看评分数据
#
# ### 2.3.1  `span.score()` 函数的使用
#
# Langfuse 支持使用 `span.score()` 函数来对 LLM 生成的输出进行评分。
# `span.score()` 函数的参数如下：
# *   `name`: 评分的名称 (必填)
# *   `value`: 评分的值 (必填，可以是数字、字符串、布尔值等)
# *   `metadata`: 评分的元数据 (可选，字典类型，可以记录额外的属性信息)
#
# ### 2.3.2 手动评分

# In[8]:
def ask_openai_with_manual_score(question):
    trace = langfuse.trace(name="手动评分的问答机器人")
    with trace.span(name="openai_call") as span:
        span.update_event(name = "user_question", input=question)
        response, latency = ask_openai(question)
        span.update_event(name="llm_response", output=response["choices"][0]["message"]["content"])
        span.log_observation(name="latency", value=latency)
        score = input("请对 AI 的回答进行评分 (1-5，5分最高): ")
        try:
            span.score(name="response_quality", value=int(score))
        except:
            print("评分格式错误，请输入数字 1-5")
    trace.end()
    return response["choices"][0]["message"]["content"]

question = "请介绍一下 Langfuse"
answer = ask_openai_with_manual_score(question)
print(f"用户提问: {question}")
print(f"AI 回答: {answer}")
# In[9]:
# ### 2.3.3 查看评分数据
#
# 运行上面的代码，并在 Langfuse 仪表板中查看结果。
# 你可以在 Span 的详细信息中找到 `response_quality` 评分，查看其 `value` 信息。
#
# **注意:**
#   *这里我们使用 input() 进行用户输入评分，在实际应用中，你可能会使用一些评分框架（会在后面的模块介绍）自动化评分。
#
#   **下一步:**
#
# *   运行上面的代码。
# *   访问 Langfuse 仪表板 (`http://localhost:3000`)，查看评分数据。
# *   尝试修改代码，记录更多的评分信息。

# In[10]:
# ## 2.4  自定义 Span 的使用
#
# **学习目标:** 掌握如何创建和使用自定义 Span 来跟踪复杂的 LLM 应用。
#
# **重点内容:**
# *   自定义 Span 的命名和层级结构
# *   在 Trace 中创建多个 Span，模拟复杂调用链
# *   在不同 Span 中记录不同的 Event 和 Observation
# *   理解 Span 的父子关系
# *   在 Langfuse 仪表板中查看自定义 Span 和调用链
#
# ### 2.4.1 自定义 Span 和层级结构
#
# 在前面的示例中，我们只使用了一个 Span 来跟踪 OpenAI 的 API 调用。
# 在实际应用中，我们可能需要创建多个 Span 来跟踪不同的操作，例如：
# *   LLM 调用前的数据预处理
# *   多个 API 调用
# *   LLM 调用后的数据处理
# *   数据库查询
#
# 我们可以使用 `trace.span()` 创建新的 Span。 Span 之间可以有父子关系，形成调用链。
# 在 Langfuse 仪表板中，我们可以看到 Span 的层级结构，方便我们理解复杂的调用链。
#
# ### 2.4.2 在 Trace 中创建多个 Span

# In[11]:
import requests

def call_api(url):
    start_time = time.time()
    response = requests.get(url)
    end_time = time.time()
    return response, end_time - start_time


def ask_openai_with_custom_spans(question):
    trace = langfuse.trace(name = "多步骤的问答机器人")
    with trace.span(name="preprocess_data") as preprocess_span:
        preprocess_span.update_event(name = "start_preprocessing", input = question)
        # 模拟数据预处理过程
        preprocessed_question = question.upper()
        preprocess_span.update_event(name = "finish_preprocessing", output = preprocessed_question)

    with trace.span(name="openai_call") as openai_span:
        openai_span.update_event(name = "user_question", input=preprocessed_question)
        response, latency = ask_openai(preprocessed_question)
        openai_span.update_event(name="llm_response", output=response["choices"][0]["message"]["content"])
        openai_span.log_observation(name="latency", value = latency)

    with trace.span(name="api_call") as api_span:
        api_span.update_event(name="api_request", input="https://dummyjson.com/products/1")
        api_response, api_latency = call_api("https://dummyjson.com/products/1")
        api_span.update_event(name = "api_response", output=api_response.json())
        api_span.log_observation(name = "latency", value = api_latency)

    trace.end()
    return response["choices"][0]["message"]["content"], api_response.json()

question = "请问我有什么产品？"
answer, api_response = ask_openai_with_custom_spans(question)
print(f"用户提问: {question}")
print(f"AI 回答: {answer}")
print(f"API 响应：{api_response}")


# In[12]:
# ### 2.4.3  查看自定义 Span 和调用链
#
# 运行上面的代码，并在 Langfuse 仪表板中查看结果。
# 你可以在 Trace 的详细信息中看到多个 Span，它们之间有层级关系，形成了一个调用链。
#  你可以点击每个 Span，查看其 Event 和 Observation。
#
#  **下一步：**
#
# *   运行上面的代码。
# *   访问 Langfuse 仪表板 (`http://localhost:3000`)，查看自定义 Span 和调用链。
# *   尝试修改代码，添加更多自定义 Span，模拟更复杂的调用流程。
#

# In[13]:
# ### 2.4.4 小结
#
# 你已经成功完成了模块二的学习，你掌握了 Langfuse 的基础功能，包括：
#  *  记录详细的 Event 信息
#  *  使用 Observation 记录性能指标
#  *  使用 Score 评估 LLM 输出质量
#  *  创建自定义 Span 跟踪复杂流程
# 你已经具备了使用 Langfuse 监控和评估简单 LLM 应用的能力。
#
# **下一步：**
#  *  巩固本模块所学的知识。
#  *  准备进入下一个模块的学习，了解更多 Langfuse 的高级功能。

Langfuse API 使用: 演示了如何使用 langfuse.traces(), langfuse.spans() 和 langfuse.observations() 查询 Langfuse 数据，并使用 pandas 进行数据分析。

与 Ragas 集成: 演示了如何集成 Ragas 评估框架，自动评估 LLM 输出质量，并将评估指标记录到 Langfuse 中。

异步操作: 演示了如何使用异步 API 进行异步操作，避免阻塞主线程，并使用 asyncio 模块进行异步编程。

Langfuse 配置进阶: 介绍了如何使用配置项和 client_options，优化 Langfuse 的性能和使用体验。

注释: 代码中包含了详细的注释，解释每个步骤的意义和目的。

In [ ]:
# -*- coding: utf-8 -*-
"""
# 模块三：Langfuse 高级 (Advanced Functionality)

这个模块将深入介绍 Langfuse 的一些高级功能，包括 Langfuse API 的使用、与 LLM 工具的集成、异步操作和 Langfuse 配置进阶。
"""

# ## 3.1  Langfuse API 的使用
#
# **学习目标:** 掌握如何使用 Langfuse Python API 查询和分析数据。
#
# **重点内容:**
# *   使用 `langfuse.traces()` 查询 Trace 数据
# *   使用 `langfuse.spans()` 查询 Span 数据
# *   使用 `langfuse.observations()` 查询 Observation 数据
# *   使用 Python 库 (如 pandas) 进行数据分析和处理
# *   根据时间范围、Trace ID 等条件查询数据
# *   使用 API 实现数据导出和自动化分析
#
# ### 3.1.1  Langfuse API 简介
#
#  Langfuse 提供了 Python API，你可以使用 API 来查询、分析和管理你的 Langfuse 数据。
#  你可以使用 API 来：
# *   获取 Trace 的详细信息
# *   获取 Span 的详细信息
# *   获取 Observation 的详细信息
# *   查询特定时间段的数据
# *   按照指定条件过滤数据
# *   导出数据到其他系统
#
#  Langfuse API 使用 RESTful 风格，并支持 JSON 格式的数据。
#  在 Python 中，你可以使用 `langfuse`  SDK 中的`client`对象来调用 Langfuse API。

# In[1]:
# 导入 Langfuse SDK
from langfuse import Langfuse
import openai
import time
import pandas as pd
from datetime import datetime, timedelta

# 初始化 Langfuse SDK
# 将 "YOUR_LANGFUSE_PUBLIC_KEY" 替换为你的 Langfuse API Key
# 将 "YOUR_LANGFUSE_HOST" 替换为你的 Langfuse Host URL， 一般是http://localhost:3000
langfuse = Langfuse(
    public_key="YOUR_LANGFUSE_PUBLIC_KEY",
    host="YOUR_LANGFUSE_HOST"
)

# 将 "YOUR_OPENAI_API_KEY" 替换为你的 OpenAI API Key
openai.api_key = "YOUR_OPENAI_API_KEY"

def ask_openai(question):
    start_time = time.time()
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": question},
        ],
    )
    end_time = time.time()
    return response, end_time - start_time

def ask_openai_with_observation(question):
    trace = langfuse.trace(name="性能指标的问答机器人")
    with trace.span(name="openai_call") as span:
        span.update_event(name = "user_question", input = question)
        response, latency = ask_openai(question)
        span.update_event(name = "llm_response", output = response["choices"][0]["message"]["content"])
        span.log_observation(name="latency", value=latency)
        span.log_observation(name="input_tokens", value=response["usage"]["prompt_tokens"], metadata={"model": "gpt-3.5-turbo"})
        span.log_observation(name="output_tokens",value = response["usage"]["completion_tokens"], metadata={"model": "gpt-3.5-turbo"})
    trace.end()
    return response["choices"][0]["message"]["content"]

question = "北京今天天气怎么样？"
answer = ask_openai_with_observation(question)
print(f"用户提问: {question}")
print(f"AI回答: {answer}")

# In[2]:
# ### 3.1.2 使用 `langfuse.traces()` 查询 Trace 数据
#
#  使用 `langfuse.traces()` 可以查询 Trace 数据。
#
#  `langfuse.traces()` 函数的参数如下：
#  *   `start_time`: 查询数据的开始时间 (可选，datetime 对象或 ISO 格式字符串)
#  *   `end_time`: 查询数据的结束时间 (可选，datetime 对象或 ISO 格式字符串)
#  *   `limit`: 查询数据的最大数量 (可选，默认值为 100)
#  *   `offset`: 查询数据的偏移量 (可选，用于分页)
#  *   `trace_id`: 查询特定的 Trace ID
#
#  `langfuse.traces()` 函数返回一个包含所有 Trace 数据的列表，每个 Trace 数据都是一个字典。

# 获取最近 24 小时的 Trace 信息
now = datetime.utcnow()
start_time = now - timedelta(days=1)

traces = langfuse.traces(
    start_time=start_time.isoformat()
).data

traces_df = pd.DataFrame(traces)
print("最近24小时的Traces信息：")
print(traces_df[["id", "name", "startTime", "endTime"]])

# In[3]:
# ### 3.1.3 使用 `langfuse.spans()` 查询 Span 数据
#
# 使用 `langfuse.spans()` 可以查询 Span 数据。
#
# `langfuse.spans()` 函数的参数与 `langfuse.traces()` 函数类似，但多了 `trace_id` 参数，用于查询特定 Trace 中的 Span 数据。

# 获取最近 24 小时的所有 Span 信息
spans = langfuse.spans(
    start_time = start_time.isoformat()
).data

spans_df = pd.DataFrame(spans)
print("\n最近24小时的Spans信息：")
print(spans_df[["id", "name", "traceId", "startTime","endTime"]])

# In[4]:
# ### 3.1.4 使用 `langfuse.observations()` 查询 Observation 数据
#
#  使用 `langfuse.observations()` 可以查询 Observation 数据。
#  `langfuse.observations()` 函数的参数与 `langfuse.traces()` 和 `langfuse.spans()` 函数类似。

# 获取最近 24 小时的所有 Observation 信息
observations = langfuse.observations(
    start_time = start_time.isoformat()
).data

observations_df = pd.DataFrame(observations)
print("\n最近24小时的Observations信息:")
print(observations_df[["id", "name", "spanId", "value","time"]])

# In[5]:
# ### 3.1.5 使用 Python 库进行数据分析
#
#   为了方便数据分析，我们可以使用 `pandas` 库将 Langfuse API 返回的数据转换为 `DataFrame`，并进行进一步的数据分析。
#  例如，我们可以计算平均延迟。

def calculate_average_latency(start_time, end_time):
  spans = langfuse.spans(
    start_time = start_time.isoformat(),
      end_time = end_time.isoformat()
  ).data
  spans_df = pd.DataFrame(spans)

  observations = langfuse.observations(
        start_time = start_time.isoformat(),
        end_time = end_time.isoformat()
    ).data

  observations_df = pd.DataFrame(observations)
  latency_observations = observations_df[observations_df["name"] == "latency"]

  merged_df = pd.merge(spans_df, latency_observations, left_on = "id", right_on = "spanId", suffixes=('_span', '_observation'))

  if not merged_df.empty:
    average_latency = merged_df["value"].mean()
    print(f"从 {start_time} 到 {end_time} 的平均延迟为: {average_latency:.4f} 秒")
  else:
    print("当前时间段内没有延迟数据")

today = datetime.utcnow()
last_week = today - timedelta(days = 7)
calculate_average_latency(last_week, today)
# In[6]:
# ### 3.1.6 更多 API 用法
#  你还可以使用 Langfuse API 进行更复杂的操作，例如：
# *  根据 Trace ID 或 Span ID 查询特定数据。
# *  根据自定义的过滤器查询数据。
# *  使用 API 导出数据。
# *  使用 API 实现自动化数据分析。
#
# 你可以参考 Langfuse 的官方文档，了解更多 API 的使用方法。
#
#  **下一步：**
#  *  运行上面的代码，查看 API 返回的数据。
#  *  尝试修改代码，使用不同的查询条件，分析不同的数据。
#  *  尝试使用 pandas 或其他数据分析工具，对 Langfuse 数据进行更深入的分析。

# In[7]:
# ## 3.2  与 LLM 工具集成
#
# **学习目标:** 掌握 Langfuse 如何与其他 LLM 工具集成，例如向量数据库、评估框架等。
#
# **重点内容:**
# *   集成向量数据库 (例如记录 LLM 调用中使用的向量数据)
# *   集成评估框架 (例如 `Ragas`, `LLMEval`)
# *   记录评估指标，实现自动化模型评估
# *   演示与常用工具集成的代码示例
#
# ### 3.2.1  与评估框架 `Ragas` 集成
#
#  为了演示 Langfuse 与评估框架的集成，我们选择 `Ragas` 评估框架。
#  `Ragas` 提供了一系列的指标，例如 `faithfulness` (忠实度), `answer_relevancy` (答案相关性),
#  `context_recall` (上下文召回率) 和 `context_precision` (上下文精度) 等指标，用于评估 LLM 生成结果的质量。
#
#  **注意:** 你需要提前安装 `ragas` 库：
#  ```bash
#  !pip install ragas
#  ```

# In[8]:
!pip install ragas

# In[9]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision

def ragas_evaluate(question, answer, context):
  eval_result = evaluate(
    [
        {
            "question": question,
            "answer": answer,
            "contexts": [context]
        }
    ],
      metrics = [faithfulness, answer_relevancy, context_recall, context_precision]
  )
  return eval_result

def ask_openai_with_ragas_and_langfuse(question, context):
    trace = langfuse.trace(name="ragas 评估的问答机器人")
    with trace.span(name="openai_call") as span:
        span.update_event(name = "user_question", input = question)
        response, latency = ask_openai(question)
        span.update_event(name="llm_response", output = response["choices"][0]["message"]["content"])
        span.log_observation(name="latency", value = latency)

        eval_result = ragas_evaluate(question, response["choices"][0]["message"]["content"], context)

        for metric_name, metric_value in eval_result.items():
            span.score(name = metric_name, value = metric_value)

    trace.end()
    return  response["choices"][0]["message"]["content"], eval_result

question = "What is the capital of France?"
context = "The capital of France is Paris."
answer, eval_result = ask_openai_with_ragas_and_langfuse(question,context)

print(f"用户提问: {question}")
print(f"AI回答: {answer}")
print(f"Ragas评估结果: {eval_result}")
# In[10]:
# ### 3.2.2 查看评估结果
#
# 运行上面的代码，并在 Langfuse 仪表板中查看结果。
#  你可以在 Span 的详细信息中找到 Ragas 评估的指标和评分。
#
#   **下一步：**
#  *  运行上面的代码，查看 Langfuse 仪表板中的结果。
#  *  尝试修改代码，集成其他评估框架或工具。

# In[11]:
# ## 3.3  异步操作
#
# **学习目标:** 掌握如何使用 Langfuse 进行异步操作，避免阻塞主线程
#
# **重点内容:**
#  *   异步 `trace()` ,`span()`, `event()` 等相关方法
#  *    异步执行 `langfuse.flush()`
#  *   使用 `asyncio` 模块实现异步编程
#
# ### 3.3.1 为什么需要异步操作
# 在实际应用中，LLM 调用可能会比较耗时，如果使用同步操作，会导致主线程阻塞，降低程序的响应速度。
# Langfuse 提供了异步操作的支持，可以让你在不阻塞主线程的情况下记录跟踪数据。
#
#  ### 3.3.2  异步操作方法
#  Langfuse 提供了异步版本的 API，例如 `langfuse.trace_async()`, `trace.span_async()`, `span.update_event_async()`, 以及 `langfuse.flush_async()` 等方法。
#  这些方法的使用方式与同步方法类似，但需要使用 `asyncio` 模块进行异步编程。
#  *   `async`：用于定义一个异步函数，这个函数可以被 `await` 调用。
#  *   `await`：用于等待一个异步函数执行完成。
#
# ### 3.3.3 异步操作示例

# In[12]:
import asyncio

async def ask_openai_async(question):
    response = await openai.ChatCompletion.acreate(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": question},
        ],
    )
    return response["choices"][0]["message"]["content"]

async def ask_openai_with_langfuse_async(question):
    trace = langfuse.trace(name="异步问答机器人")
    async with trace.span(name="openai_call") as span:
      await span.update_event_async(name="user_question", input = question)
      response = await ask_openai_async(question)
      await span.update_event_async(name = "llm_response", output=response)
    await trace.end_async()
    await langfuse.flush_async()
    return response

async def main():
  question = "你好，异步操作怎么样？"
  answer = await ask_openai_with_langfuse_async(question)
  print(f"用户提问: {question}")
  print(f"AI回答: {answer}")

asyncio.run(main())


# In[13]:
# ### 3.3.4 注意
#  *  使用异步 API 时，需要使用 `await` 关键字等待异步操作完成。
#  *  需要在 `asyncio.run()` 中执行异步函数。
#  *  需要调用 `langfuse.flush_async()` 确保所有数据都被正确提交。
#
#  **下一步:**
#  *   运行上面的代码，查看 Langfuse 仪表板中的结果。
#  *   尝试修改代码，使用异步方法记录更多信息。

# In[14]:
# ## 3.4  Langfuse 配置进阶
#
# **学习目标:** 掌握Langfuse的配置项，优化Langfuse使用体验
#
# **重点内容:**
#  *  使用 `debug`, `timeout`, `batch_size` 等参数优化性能
#  *  使用 `client_options` 配置HTTP请求头等信息
#  *   自定义 Langfuse SDK 的行为和输出
#
# ### 3.4.1 配置参数
# 在初始化 Langfuse SDK 时，你可以设置一些参数来优化性能。
# *   `debug`： 开启调试模式，输出详细日志信息
# *   `timeout`： 设置 HTTP 请求超时时间，单位秒
# *   `batch_size`：设置批量提交数据的数量，可以提升性能
#
# ```python
# langfuse = Langfuse(
#    public_key="YOUR_LANGFUSE_PUBLIC_KEY",
#    host="YOUR_LANGFUSE_HOST",
#    debug=True,
#    timeout=10,
#    batch_size=100
#  )
# ```
# ### 3.4.2 client_options 配置HTTP请求
# `client_options` 可以帮助你自定义HTTP请求头，例如添加 `Authorization` 等
#
# ```python
# langfuse = Langfuse(
#    public_key="YOUR_LANGFUSE_PUBLIC_KEY",
#    host="YOUR_LANGFUSE_HOST",
#    client_options = {"headers": {"Authorization": "Bearer XXXX"}}
#  )
# ```
# ### 3.4.3 自定义 Langfuse SDK 行为
# 你可以通过继承 Langfuse 的 `BaseLangfuse` 类，自定义 Langfuse SDK 的行为。
#
#  例如，你可以修改 SDK 提交数据的方式、调整日志输出、自定义错误处理等。

#  **下一步:**
#  *  修改初始化代码，尝试配置不同的参数。
#  *  阅读 Langfuse 文档，了解更多配置项。

# In[15]:
# ### 3.4.4 小结
#
# 你已经成功完成了模块三的学习，你掌握了 Langfuse 的高级功能，包括：
# * 使用 Langfuse API 查询分析数据
# * 集成 Ragas 等评估框架
# * 使用异步操作
# * Langfuse 配置进阶
#  你已经具备了使用 Langfuse 分析复杂的 LLM 应用，并与其他工具集成。
#
# **下一步:**
# * 巩固本模块所学的知识。
# * 准备进入下一个模块的学习，掌握 Langfuse 的实战技巧。

复杂 LLM 应用监控: 演示了如何使用 Langfuse 监控复杂的 LLM 应用，并结合实际代码演示了如何记录多个 Span 和 Event，使用 Observation 监控性能指标，使用评分功能评估 LLM 输出。

优化 LLM 应用: 详细介绍了如何使用 Langfuse 分析数据，并基于分析结果优化 LLM 应用，包括 LLM 模型参数、Prompt Engineering等。

构建可观测的 LLM 代理: 演示了如何使用 Langfuse 构建可观测的 LLM 代理，并记录代理的决策过程、执行步骤、性能指标等。

最佳实践和进阶技巧: 总结了使用 Langfuse 的最佳实践，包括如何清晰命名 Span 和 Event、如何合理使用 Metadata、如何有效利用 Langfuse 仪表板、如何与其他工具集成 Langfuse 等。

注释: 代码中包含了详细的注释，解释每个步骤的意义和目的。

In [ ]:
# -*- coding: utf-8 -*-
"""
# 模块四：Langfuse 实战 (Real-World Applications)

这个模块将引导你如何使用 Langfuse 来监控复杂的 LLM 应用，进行性能优化，构建可观测的 LLM 代理，并总结 Langfuse 的最佳实践。
"""

# ## 4.1  监控复杂的 LLM 应用
#
# **学习目标:** 能够使用 Langfuse 监控复杂的 LLM 应用，并找出潜在问题。
#
# **重点内容:**
# *   设计合适的 Span 和 Event 结构，跟踪复杂调用链
# *   使用 Observation 监控性能指标，并找出瓶颈
# *   使用评分功能评估 LLM 输出质量，并找出问题
# *   利用 Langfuse API 分析数据，找出潜在的性能或质量问题
#
# ### 4.1.1 复杂 LLM 应用的特点
#
# 复杂的 LLM 应用通常具有以下特点：
# *   多个步骤： 需要调用多个 API 或执行多个操作。
# *   多个 LLM 模型： 可能需要调用多个 LLM 模型。
# *   复杂的逻辑：  需要进行数据处理、条件判断、逻辑控制等。
# *   异步操作：  需要处理异步任务，例如后台处理、消息队列等。
#
# ### 4.1.2 设计合适的 Span 和 Event 结构
#
# 对于复杂的 LLM 应用，我们需要设计合适的 Span 和 Event 结构，清晰地记录应用的执行流程。
# *   **Span 的粒度：** Span 的粒度应该根据实际情况进行调整，不要过于粗略，也不要过于细致。一般来说，一个 Span 应该代表一个有意义的操作单元。
# *   **Event 的信息：** Event 应该记录足够多的信息，例如输入、输出、参数、状态、错误信息等，方便后续分析。
# *   **Span 的层级结构：** 合理利用 Span 的层级结构，清晰地记录调用链，方便理解应用的执行流程。
#
# ### 4.1.3 使用 Observation 监控性能指标
#
# 对于复杂的 LLM 应用，我们需要关注一些关键的性能指标，例如：
# *   总延迟：  整个 Trace 的执行时间。
# *   每个 Span 的延迟：  每个 Span 的执行时间。
# *   LLM 模型的延迟： LLM 模型的调用时间。
# *   API 调用的延迟： API 接口的调用时间。
# *   数据库查询的延迟： 数据库查询的时间。
# *   Token 数：  LLM 模型的输入和输出 Token 数。
#
#  通过监控这些性能指标，我们可以找出潜在的性能瓶颈，例如慢的 API 调用、耗时的数据库查询等。
#
# ### 4.1.4 使用评分功能评估 LLM 输出质量
#
#  对于复杂的 LLM 应用，我们需要关注 LLM 输出的质量，例如：
# *   相关性：  LLM 输出是否与输入相关。
# *   准确性：  LLM 输出是否准确。
# *   完整性：  LLM 输出是否完整。
# *   逻辑性：  LLM 输出是否逻辑清晰。
# *   安全性： LLM 输出是否安全。
#
# 通过评分功能，我们可以评估 LLM 输出的质量，并找出潜在的问题。
#
# ### 4.1.5 使用 Langfuse API 分析数据
#
#  对于复杂的 LLM 应用，我们需要利用 Langfuse API 分析数据，找出潜在的问题。
#  例如，我们可以：
# *   统计每个 Span 的平均延迟。
# *   统计每个 Event 的发生次数。
# *   分析 LLM 输出的评分分布。
# *   找出执行时间最长的 Trace。
# *   找出评分最低的 Trace。
#
# 通过分析数据，我们可以找出潜在的性能或质量问题，为优化提供依据。

# In[1]:
# 导入 Langfuse SDK
from langfuse import Langfuse
import openai
import time
import pandas as pd
from datetime import datetime, timedelta
import requests
import asyncio

# 初始化 Langfuse SDK
# 将 "YOUR_LANGFUSE_PUBLIC_KEY" 替换为你的 Langfuse API Key
# 将 "YOUR_LANGFUSE_HOST" 替换为你的 Langfuse Host URL， 一般是http://localhost:3000
langfuse = Langfuse(
    public_key="YOUR_LANGFUSE_PUBLIC_KEY",
    host="YOUR_LANGFUSE_HOST"
)

# 将 "YOUR_OPENAI_API_KEY" 替换为你的 OpenAI API Key
openai.api_key = "YOUR_OPENAI_API_KEY"

def ask_openai(question):
    start_time = time.time()
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": question},
        ],
    )
    end_time = time.time()
    return response, end_time - start_time

def call_api(url):
    start_time = time.time()
    response = requests.get(url)
    end_time = time.time()
    return response, end_time - start_time
# In[2]:
# ### 4.1.6 复杂的LLM应用示例

def complex_llm_app(question):
    trace = langfuse.trace(name="复杂LLM应用")
    with trace.span(name="preprocess") as preprocess_span:
        preprocess_span.update_event(name="start_preprocess", input = question)
        preprocessed_question = question.upper()
        preprocess_span.update_event(name="end_preprocess", output = preprocessed_question)

    with trace.span(name="openai_call") as openai_span:
        openai_span.update_event(name="user_question", input = preprocessed_question)
        response, latency = ask_openai(preprocessed_question)
        openai_span.update_event(name="llm_response", output = response["choices"][0]["message"]["content"])
        openai_span.log_observation(name="latency", value = latency)
        openai_span.log_observation(name="input_tokens", value = response["usage"]["prompt_tokens"])
        openai_span.log_observation(name = "output_tokens", value = response["usage"]["completion_tokens"])

    with trace.span(name="api_call") as api_span:
        api_span.update_event(name="api_request", input="https://dummyjson.com/products/1")
        api_response, api_latency = call_api("https://dummyjson.com/products/1")
        api_span.update_event(name="api_response", output = api_response.json())
        api_span.log_observation(name = "latency", value = api_latency)

    with trace.span(name="postprocess") as postprocess_span:
         postprocess_span.update_event(name="start_postprocess",input= response["choices"][0]["message"]["content"])
         # 模拟一些后处理
         postprocessed_response = response["choices"][0]["message"]["content"].lower()
         postprocess_span.update_event(name="end_postprocess",output=postprocessed_response)

    score = input("请对 AI 的回答进行评分 (1-5，5分最高): ")
    try:
      trace.score(name="response_quality", value=int(score))
    except:
      print("评分格式错误，请输入数字 1-5")

    trace.end()
    return response["choices"][0]["message"]["content"], api_response.json()


question = "请问我有什么产品，并且用小写的形式给我？"
answer, api_response = complex_llm_app(question)
print(f"用户提问: {question}")
print(f"AI 回答: {answer}")
print(f"API 响应：{api_response}")

# In[3]:
# ### 4.1.7 使用 Langfuse API 分析数据
# 获取最近 24 小时的所有 Span 信息
now = datetime.utcnow()
start_time = now - timedelta(days=1)
spans = langfuse.spans(
    start_time = start_time.isoformat()
).data

spans_df = pd.DataFrame(spans)
print("\n最近24小时的Spans信息：")
print(spans_df[["id", "name", "traceId", "startTime","endTime"]])

# 计算每个 Span 的平均延迟

def calculate_average_latency_for_spans(start_time, end_time):
  spans = langfuse.spans(
    start_time = start_time.isoformat(),
      end_time = end_time.isoformat()
  ).data
  spans_df = pd.DataFrame(spans)

  observations = langfuse.observations(
        start_time = start_time.isoformat(),
        end_time = end_time.isoformat()
    ).data

  observations_df = pd.DataFrame(observations)
  latency_observations = observations_df[observations_df["name"] == "latency"]

  merged_df = pd.merge(spans_df, latency_observations, left_on = "id", right_on = "spanId", suffixes=('_span', '_observation'))

  if not merged_df.empty:
    average_latency = merged_df.groupby("name_span")["value"].mean().reset_index()
    print(f"从 {start_time} 到 {end_time} 的每个 Span 的平均延迟为:")
    print(average_latency)
  else:
    print("当前时间段内没有延迟数据")


today = datetime.utcnow()
last_week = today - timedelta(days = 7)
calculate_average_latency_for_spans(last_week, today)

#  **下一步：**
#  *  运行上面的代码，查看 Langfuse 仪表板中的结果。
#  *  尝试修改代码，记录更多的信息。
#  *  尝试使用 Langfuse API 分析数据，找出潜在的性能或质量问题。

# In[4]:
# ## 4.2  优化 LLM 应用
#
# **学习目标:** 能够使用 Langfuse 来指导 LLM 应用的优化过程。
#
# **重点内容:**
# *   根据性能指标和评估结果，优化 LLM 模型或参数
# *   调整 Prompt Engineering, 提升LLM输出质量
# *   使用 Langfuse 监控优化效果，并进行迭代改进
#
# ### 4.2.1  基于 Langfuse 的优化流程
#  使用 Langfuse 优化 LLM 应用的流程通常如下：
#  1.  **监控：** 使用 Langfuse 监控你的 LLM 应用，记录 Trace, Span, Event, Observation 和 Score。
#  2.  **分析：** 使用 Langfuse 仪表板或 API 分析数据，找出潜在的性能或质量问题。
#  3.  **优化：** 根据分析结果，优化你的 LLM 应用，例如修改 Prompt, 修改模型参数，调整调用逻辑。
#  4.  **测试：** 使用新的代码重新测试你的 LLM 应用，并使用 Langfuse 记录新的数据。
#  5.  **迭代：** 分析新的数据，判断优化效果，重复优化步骤。
#
# ### 4.2.2  根据性能指标优化
#
#   * **平均延迟：** 如果平均延迟过高，则需要分析是哪个 Span 耗时最长，可能是 API 调用、模型计算、数据库查询等。
#       *  优化：可以优化慢的 API， 使用更快的 LLM 模型，或使用数据库缓存。
#   * **LLM 延迟：** 如果 LLM 延迟过高，则需要尝试使用更快的 LLM 模型，或减少请求的 Token 数。
#       *  优化： 使用更快的 LLM 模型，或者使用 Prompt Engineering 减少 token 数量。
#   * **Token 数：** 如果 Token 数过高，则会增加成本，可以考虑优化 Prompt, 减少 Token 数。
#      *  优化： 可以使用更简短的 Prompt，或者使用更适合的 LLM 模型。
#
# ### 4.2.3  根据评估结果优化
#  * **相关性：** 如果 LLM 的输出相关性过低，可以尝试调整 Prompt, 提升 LLM 的理解能力。
#      * 优化： 可以使用更清晰的 Prompt，或者使用多轮对话来引导 LLM 输出。
#  * **准确性：** 如果 LLM 的输出准确性过低，可以尝试使用更专业的 LLM 模型，或者优化模型训练数据。
#      * 优化： 可以使用更专业的 LLM 模型，或增加 Few Shot 示例。
#  *  **安全性：** 如果 LLM 的输出安全性过低，可以尝试使用安全策略，避免产生有害或不安全的内容。
#      * 优化： 可以使用 Prompt Engineering，或使用安全策略来过滤LLM输出。
#
# ### 4.2.4  使用 Langfuse 监控优化效果
#
#  在优化之后，你需要使用 Langfuse 监控你的 LLM 应用，并判断优化效果。
#  *   使用 Langfuse 仪表板查看优化后的性能指标和评估结果。
#  *   使用 Langfuse API 分析优化后的数据，并与优化前的数据进行对比。
#  *  进行多次迭代优化，直到满足你的要求。
#
# **下一步:**
#  *  根据你的实际情况，使用 Langfuse 分析你的 LLM 应用，并进行优化。
#  *  使用 Langfuse 监控优化效果，并进行迭代改进。

# In[5]:
# ## 4.3  构建可观测的 LLM 代理
#
# **学习目标:** 掌握如何使用 Langfuse 来构建可观测的 LLM 代理。
#
# **重点内容:**
# *   使用 Langfuse 跟踪 LLM 代理的执行过程
# *   记录代理的决策过程，便于调试和分析
# *   使用 Langfuse 评估代理的性能和效果
# *   监控代理的调用次数、耗时等关键指标
#
# ### 4.3.1  LLM 代理的定义
#
#   LLM 代理是指能够自主执行任务的 LLM 程序，它通常具有以下特点：
#  *  具有一定的记忆能力。
#  *  能够自主进行决策。
#  *  能够调用外部工具或 API。
#  *  能够与用户进行多轮交互。
#
# ### 4.3.2 使用 Langfuse 跟踪 LLM 代理
#
#   使用 Langfuse 跟踪 LLM 代理的执行过程，需要记录以下信息：
#  *   代理的执行步骤 (使用 Span 来记录)。
#  *   代理的决策过程 (使用 Event 记录)。
#  *   代理调用的工具或 API (使用 Event 记录)。
#  *   代理的输入和输出 (使用 Event 记录)。
#  *   代理的执行耗时 (使用 Observation 记录)。
#
# ### 4.3.3 记录代理的决策过程
#  LLM 代理的决策过程通常是比较复杂的，需要在多个步骤进行决策。
#  可以使用 Langfuse 的 Event 来记录代理的决策过程，例如：
#  *   代理选择使用的工具。
#  *   代理选择执行的操作。
#  *   代理的中间状态。
#  通过记录这些信息，可以帮助你了解代理的决策逻辑，并进行调试和分析。
#
#  ### 4.3.4 使用 Langfuse 评估代理的性能
#
#  使用 Langfuse 评估代理的性能，需要记录以下指标：
#  *  任务完成率： 代理成功完成任务的比例。
#  *  任务完成时间： 代理完成任务的时间。
#  *  资源消耗：  代理的 CPU, 内存等资源消耗。
#  *  成本：  代理的调用成本。
#
# ### 4.3.5 LLM 代理示例
#  假设我们有一个 LLM 代理，它的任务是使用 OpenAI 和一个API获取相关信息，并输出最终结果。

# In[6]:
async def ask_openai_async(question):
    response = await openai.ChatCompletion.acreate(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": question},
        ],
    )
    return response, time.time()


async def call_api_async(url):
    start_time = time.time()
    response = requests.get(url)
    end_time = time.time()
    return response, end_time - start_time

async def complex_agent(question):
    trace = langfuse.trace(name="LLM 代理")

    async with trace.span(name="agent_decision") as decision_span:
        decision_span.update_event(name="user_question", input=question)
        decision_span.update_event(name="agent_decide_step", output="需要调用 openai 获取相关信息")
        openai_response, openai_time = await ask_openai_async(question)
        await decision_span.update_event_async(name="openai_response", output=openai_response["choices"][0]["message"]["content"])
        decision_span.log_observation(name="openai_latency", value=openai_time - time.time() )
        await decision_span.update_event_async(name="agent_decide_step", output="需要调用 API 获取产品信息")

    async with trace.span(name="api_call") as api_span:
        api_span.update_event(name="api_request", input="https://dummyjson.com/products/1")
        api_response, api_latency = await call_api_async("https://dummyjson.com/products/1")
        await api_span.update_event_async(name="api_response", output=api_response.json())
        api_span.log_observation(name="latency", value = api_latency)


    await trace.end_async()
    await langfuse.flush_async()

    return openai_response["choices"][0]["message"]["content"], api_response.json()

async def main():
  question = "请问我有什么产品？"
  openai_answer, api_response = await complex_agent(question)
  print(f"用户提问: {question}")
  print(f"AI 回答: {openai_answer}")
  print(f"API 响应：{api_response}")

asyncio.run(main())

# In[7]:
# ### 4.3.6 查看结果
#  运行上面的代码，并在 Langfuse 仪表板中查看结果。 你可以看到 LLM 代理的执行过程、决策过程、调用的工具、输入输出、性能指标等等。
#
#   **下一步：**
#  *  运行上面的代码，查看 Langfuse 仪表板中的结果。
#  *  尝试修改代码，记录更多的代理信息。

# In[8]:
# ## 4.4  最佳实践和进阶技巧
#
# **学习目标:** 掌握 Langfuse 的最佳实践，提高使用效率和效果。
#
# **重点内容:**
# *   如何清晰命名 Span 和 Event
# *   如何合理使用 metadata
# *   如何有效利用 Langfuse 仪表板
# *   如何与其他工具集成 Langfuse
# *   如何解决使用 Langfuse 过程中的常见问题
#
# ### 4.4.1 如何清晰命名 Span 和 Event
#
#  清晰命名 Span 和 Event 是非常重要的，它可以让你更方便地理解和分析数据。
#  *   Span 的命名： 应该能够清晰地表达 Span 的功能，例如 "openai_call", "api_call", "preprocess_data" 等。
#  *   Event 的命名： 应该能够清晰地表达 Event 的类型，例如 "user_question", "llm_response", "api_request" 等。
#  *   避免使用模糊或不清晰的命名，例如 "step1", "step2" 等。
#
# ### 4.4.2 如何合理使用 Metadata
#
#   Metadata 可以记录额外的属性信息，例如用户 ID, 模型版本，请求参数，响应头等。
#  *   合理使用 Metadata 可以帮助你更全面地了解数据，并进行更深入的分析。
#  *   避免在 Metadata 中记录过多的敏感信息，例如用户密码、个人隐私等。
#  *   Metadata 的数据类型应该尽量保持一致。
#
# ### 4.4.3 如何有效利用 Langfuse 仪表板
#
#  Langfuse 仪表板提供了可视化的界面，可以方便地查看和分析数据。
#  *  使用仪表板中的过滤器，可以根据不同的条件过滤数据。
#  * 使用仪表板中的图表，可以直观地了解数据的分布和趋势。
#  * 使用仪表板中的搜索功能，可以快速找到你想要的数据。
#  *  自定义仪表板，根据自己的需求配置展示的数据。
#
# ### 4.4.4 如何与其他工具集成 Langfuse
#
#  Langfuse 可以与许多其他的工具集成，例如：
#  *   向量数据库 (例如记录 LLM 调用中使用的向量数据)
#  *   评估框架 (例如 `Ragas`, `LLMEval`)
#  *   日志系统 (例如将 Langfuse 的数据同步到你的日志系统中)
#  *   监控系统 (例如将 Langfuse 的数据同步到你的监控系统中)
#
#  通过与其他工具集成，可以提高你的开发效率，并使你的 LLM 应用更加强大。
#
# ### 4.4.5 如何解决使用 Langfuse 过程中的常见问题
#
#  *  仔细查看 Langfuse 的官方文档，解决大部分问题。
#  *  检查你的 API Key 和 Host URL 是否配置正确。
#  *  查看你的网络连接是否正常。
#  *  尝试使用 `debug` 模式，查看详细的日志输出。
#  *  如果问题仍然无法解决，请向 Langfuse 社区寻求帮助。

# In[8]:
# ### 4.4.6 小结
#
# 你已经成功完成了模块四的学习，你掌握了 Langfuse 的实战技巧，包括：
#  *   监控复杂的 LLM 应用
#  *   优化 LLM 应用
#  *   构建可观测的 LLM 代理
#  *   Langfuse 最佳实践
#
# 你已经具备了使用 Langfuse 开发和优化 LLM 应用的能力。
#
#  **下一步：**
#  *  巩固本模块所学的知识。
#  *  在你的实际项目中使用 Langfuse，并不断改进和优化。
#  *  参与 Langfuse 社区，贡献你的代码和经验。